In [2]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [3]:
adeft_grounder = AdeftGrounder()

In [4]:
shortforms = ['MV']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [5]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('measles virus', 626),
 ('mechanical ventilation', 519),
 ('methyl viologen', 153),
 ('mitral valve', 108),
 ('microvesicles', 104),
 ('membrane vesicles', 33),
 ('measles vaccine', 28),
 ('matrix vesicles', 28),
 ('minute ventilation', 23),
 ('minute volume', 19),
 ('methylviologen', 18),
 ('microvilli', 16),
 ('microvascular', 14),
 ('mother vessels', 12),
 ('myxoma virus', 12),
 ('mean velocity', 11),
 ('megavoltage', 10),
 ('microvillous columnar epithelial cells', 10),
 ('microvessels', 8),
 ('mature virus', 8),
 ('malignant rabbit fibroma virus', 8),
 ('methyl violet', 7),
 ('mycobacterium vaccae', 5),
 ('mature virions', 5),
 ('main vessels', 5),
 ('micturition volume', 5),
 ('multivitamin', 5),
 ('mumps virus', 5),
 ('methyl vanillate', 4),
 ('myosin v', 4),
 ('mechanical vibration', 4),
 ('mesenteric veins', 4),
 ('microvillar', 4),
 ('mopeia virus', 4),
 ('mixed valence', 3),
 ('muscle vibration', 3),
 ('maximum velocity', 3),
 ('metavinculin', 3),
 ('multivessel', 3),
 ('m

In [6]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

INFO: [2020-09-30 03:31:16] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.2/bio_ontology.pkl


In [7]:
result = [grounding_map, names, pos_labels]

In [8]:
result

[{'ma and veins': 'ungrounded',
  'main vessels': 'ungrounded',
  'malignant rabbit fibroma virus': 'MESH:D012776',
  'marseilleviridae': 'ungrounded',
  'matrix vesicles': 'ungrounded',
  'mature virions': 'MESH:D014771',
  'mature virus': 'ungrounded',
  'maximum velocity': 'ungrounded',
  'mean velocity': 'ungrounded',
  'measles vaccine': 'MESH:D008458',
  'measles virus': 'MESH:D008459',
  'mechanical ventilation': 'MESH:D012121',
  'mechanical vibration': 'MESH:D014732',
  'megavoltage': 'ungrounded',
  'membrane vesicles': 'ungrounded',
  'mesenteric veins': 'ungrounded',
  'metavinculin': 'MESH:C103334',
  'methyl vanillate': 'CHEBI:CHEBI:46477',
  'methyl violet': 'CHEBI:CHEBI:41688',
  'methyl viologen': 'CHEBI:CHEBI:34905',
  'methyl viologen dichloride hydrate': 'ungrounded',
  'methylviologen': 'CHEBI:CHEBI:34905',
  'microvascular': 'MESH:D055806',
  'microvesicles': 'MESH:D055252',
  'microvessels': 'MESH:D055806',
  'microvillar': 'GO:GO:0005902',
  'microvilli': 'GO:GO

In [6]:
grounding_map, names, pos_labels = [{'ma and veins': 'ungrounded',
  'main vessels': 'ungrounded',
  'malignant rabbit fibroma virus': 'MESH:D012776',
  'marseilleviridae': 'ungrounded',
  'matrix vesicles': 'ungrounded',
  'mature virions': 'MESH:D014771',
  'mature virus': 'ungrounded',
  'maximum velocity': 'ungrounded',
  'mean velocity': 'ungrounded',
  'measles vaccine': 'MESH:D008458',
  'measles virus': 'MESH:D008459',
  'mechanical ventilation': 'MESH:D012121',
  'mechanical vibration': 'MESH:D014732',
  'megavoltage': 'ungrounded',
  'membrane vesicles': 'ungrounded',
  'mesenteric veins': 'ungrounded',
  'metavinculin': 'HGNC:12665',
  'meta vinculin': 'HGNC:12665',
  'methyl vanillate': 'CHEBI:CHEBI:46477',
  'methyl violet': 'CHEBI:CHEBI:41688',
  'methyl viologen': 'CHEBI:CHEBI:34905',
  'methyl viologen dichloride hydrate': 'ungrounded',
  'methylviologen': 'CHEBI:CHEBI:34905',
  'microvascular': 'MESH:D055806',
  'microvesicles': 'MESH:D055252',
  'microvessels': 'MESH:D055806',
  'microvillar': 'GO:GO:0005902',
  'microvilli': 'GO:GO:0005902',
  'microvillous columnar epithelial cells': 'ungrounded',
  'micturition volume': 'ungrounded',
  'minute ventilation': 'MESH:D014691',
  'minute volume': 'NCIT:C120936',
  'mitral valve': 'MESH:D008943',
  'mixed valence': 'ungrounded',
  'mopeia virus': 'ungrounded',
  'mother vessels': 'ungrounded',
  'multivariate': 'ungrounded',
  'multivessel': 'ungrounded',
  'multivitamin': 'ungrounded',
  'mumps virus': 'MESH:D009109',
  'muscle vibration': 'ungrounded',
  'mv infection vero cells': 'MESH:D014709',
  'mycobacterium vaccae': 'MESH:C000641325',
  'myosin v': 'MESH:D024701',
  'myxoma virus': 'MESH:D009233'},
 {'MESH:D012776': 'Fibroma Virus, Rabbit',
  'MESH:D014771': 'Virion',
  'MESH:D008458': 'Measles Vaccine',
  'MESH:D008459': 'Measles virus',
  'MESH:D012121': 'Respiration, Artificial',
  'MESH:D014732': 'Vibration',
  'HGNC:12665': 'VCL',
  'CHEBI:CHEBI:46477': 'methyl vanillate',
  'CHEBI:CHEBI:41688': 'crystal violet',
  'CHEBI:CHEBI:34905': 'paraquat',
  'MESH:D055806': 'Microvessels',
  'MESH:D055252': 'Cell-Derived Microparticles',
  'GO:GO:0005902': 'microvillus',
  'MESH:D014691': 'Ventilation',
  'NCIT:C120936': 'Minute Volume',
  'MESH:D008943': 'Mitral Valve',
  'MESH:D009109': 'Mumps virus',
  'MESH:D014709': 'Vero Cells',
  'MESH:C000641325': 'Mycolicibacterium vaccae',
  'MESH:D024701': 'Myosin Type V',
  'MESH:D009233': 'Myxoma virus'},
 ['CHEBI:CHEBI:34905',
  'GO:GO:0005902',
  'MESH:D008458',
  'MESH:D008459',
  'MESH:D008943',
  'MESH:D012121',
  'MESH:D055252',
  'MESH:D055806',
  'HGNC:12665']]

In [7]:
excluded_longforms = []

In [8]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [9]:
additional_entities = {'HGNC:12665': ['VCL', ['MV']]}

In [10]:
unambiguous_agent_texts = {}

In [11]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [13]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [14]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [15]:
intersection1

[('HGNC:12665', 'HGNC:12665', 680)]

In [16]:
intersection2

[('MESH:D008459', 'HGNC:12665', 0),
 ('GO:GO:0005902', 'HGNC:12665', 0),
 ('ungrounded', 'HGNC:12665', 0),
 ('NCIT:C120936', 'HGNC:12665', 0),
 ('MESH:D009109', 'HGNC:12665', 0),
 ('CHEBI:CHEBI:34905', 'HGNC:12665', 0),
 ('MESH:D055806', 'HGNC:12665', 0),
 ('MESH:D012776', 'HGNC:12665', 0),
 ('MESH:D012121', 'HGNC:12665', 0),
 ('MESH:D014709', 'HGNC:12665', 0),
 ('MESH:D008943', 'HGNC:12665', 0),
 ('MESH:D009233', 'HGNC:12665', 0),
 ('MESH:D014691', 'HGNC:12665', 0),
 ('MESH:D014732', 'HGNC:12665', 0),
 ('MESH:C000641325', 'HGNC:12665', 0),
 ('MESH:D024701', 'HGNC:12665', 0),
 ('MESH:D008458', 'HGNC:12665', 0),
 ('MESH:D055252', 'HGNC:12665', 0),
 ('MESH:D014771', 'HGNC:12665', 0),
 ('HGNC:12665', 'HGNC:12665', 2),
 ('CHEBI:CHEBI:41688', 'HGNC:12665', 0),
 ('CHEBI:CHEBI:46477', 'HGNC:12665', 0)]

In [17]:
2

2

In [12]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [13]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [14]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-30 05:10:56] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-30 05:11:42] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9285990887280191 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [15]:
classifier.stats

{'label_distribution': {'MESH:D008459': 489,
  'GO:GO:0005902': 16,
  'ungrounded': 102,
  'NCIT:C120936': 13,
  'MESH:D009109': 3,
  'CHEBI:CHEBI:34905': 120,
  'MESH:D055806': 16,
  'MESH:D012776': 6,
  'MESH:D012121': 349,
  'MESH:D014709': 1,
  'MESH:D008943': 81,
  'MESH:D009233': 10,
  'MESH:D014691': 17,
  'MESH:D014732': 4,
  'MESH:C000641325': 3,
  'MESH:D024701': 4,
  'MESH:D008458': 14,
  'MESH:D055252': 69,
  'MESH:D014771': 5,
  'HGNC:12665': 3,
  'CHEBI:CHEBI:41688': 4,
  'CHEBI:CHEBI:46477': 2},
 'f1': {'mean': 0.928599, 'std': 0.013841},
 'precision': {'mean': 0.912195, 'std': 0.016961},
 'recall': {'mean': 0.952478, 'std': 0.011203},
 'NCIT:C120936': {'f1': {'mean': 0.16, 'std': 0.32},
  'pr': {'mean': 0.133333, 'std': 0.266667},
  'rc': {'mean': 0.2, 'std': 0.4}},
 'MESH:D009109': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'MESH:D055806': {'f1': {'mean': 0.773333, 'std': 0.225487},
  'pr': {'mean': 0.71666

In [16]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [17]:
disamb.dump(model_name, results_path)

In [18]:
print(disamb.info())

Disambiguation model for MV

Produces the disambiguations:
	Cell-Derived Microparticles*	MESH:D055252
	Fibroma Virus, Rabbit	MESH:D012776
	Measles Vaccine*	MESH:D008458
	Measles virus*	MESH:D008459
	Microvessels*	MESH:D055806
	Minute Volume	NCIT:C120936
	Mitral Valve*	MESH:D008943
	Mumps virus	MESH:D009109
	Mycolicibacterium vaccae	MESH:C000641325
	Myosin Type V	MESH:D024701
	Myxoma virus	MESH:D009233
	Respiration, Artificial*	MESH:D012121
	VCL*	HGNC:12665
	Ventilation	MESH:D014691
	Vero Cells	MESH:D014709
	Vibration	MESH:D014732
	Virion	MESH:D014771
	crystal violet	CHEBI:CHEBI:41688
	methyl vanillate	CHEBI:CHEBI:46477
	microvillus*	GO:GO:0005902
	paraquat*	CHEBI:CHEBI:34905

Class level metrics:
--------------------
Grounding                  	Count	F1     
              Measles virus*	489	0.94242
    Respiration, Artificial*	349	0.96116
                   paraquat*	120	0.93834
                 Ungrounded	102	0.73852
               Mitral Valve*	 81	0.96845
Cell-Derived Microparticles

In [19]:
print(disamb.info())

Disambiguation model for MV

Produces the disambiguations:
	Cell-Derived Microparticles*	MESH:D055252
	Fibroma Virus, Rabbit	MESH:D012776
	Measles Vaccine*	MESH:D008458
	Measles virus*	MESH:D008459
	Microvessels*	MESH:D055806
	Minute Volume	NCIT:C120936
	Mitral Valve*	MESH:D008943
	Mumps virus	MESH:D009109
	Mycolicibacterium vaccae	MESH:C000641325
	Myosin Type V	MESH:D024701
	Myxoma virus	MESH:D009233
	Respiration, Artificial*	MESH:D012121
	VCL*	HGNC:12665
	Ventilation	MESH:D014691
	Vero Cells	MESH:D014709
	Vibration	MESH:D014732
	Virion	MESH:D014771
	crystal violet	CHEBI:CHEBI:41688
	methyl vanillate	CHEBI:CHEBI:46477
	microvillus*	GO:GO:0005902
	paraquat*	CHEBI:CHEBI:34905

Class level metrics:
--------------------
Grounding                  	Count	F1     
              Measles virus*	489	0.94242
    Respiration, Artificial*	349	0.96116
                   paraquat*	120	0.93834
                 Ungrounded	102	0.73852
               Mitral Valve*	 81	0.96845
Cell-Derived Microparticles

In [20]:
model_to_s3(disamb)

In [21]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [22]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:12665']

In [23]:
len(texts)

2

In [24]:
print('\n\n'.join(texts))

Vinculin and its splice variant, metavinculin (MV), are key elements of multiple protein assemblies linking the extracellular matrix to the actin cytoskeleton. Vinculin is expressed ubiquitously, whereas MV is mainly expressed in smooth and cardiac muscle tissue. The only difference in amino acid sequence between the isoforms is a 68-residue insert in the C-terminal tail domain of MV (MVt). Although the functional role of this insert remains elusive, its importance is exemplified by point mutations that are associated with dilated and hypertrophic cardiomyopathy. In vinculin, the actin binding site resides in the tail domain. In this paper, we show that MVt binds actin filaments similarly to the vinculin tail domain. Unlike its splice variant, MVt did not bundle actin filaments. Instead, MVt promoted severing of actin filaments, most efficiently at substoichiometric concentrations. This surprising and seemingly contradictory alteration of vinculin function by the 68-residue insert may 